#### Chrysanthos Lianos
#### MSc Data Science 2016 Part Time
#### Machine Learning and Computational statistics Project

### First Part

In [1]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as image
import pandas as pd
import scipy 
%matplotlib inline
#initialize the onehot encoder
encoder = OneHotEncoder(dtype='int') 

In [2]:
def getdf(type):
    '''returns all rows in train_ or test_, depending on input'''
    tables = []
    for i in range(10):
        temp_df = pd.read_csv('D:/mnistdata/{}{}.txt'.format(str(type),i),sep = ' ', names = range(784))
        temp_df['target'] = i
        tables.append(temp_df)
    tables = pd.concat(tables)
    return tables

Now get the data
train_X is the set of training data, test_X is test
train_Y is the OneHot column for train_X and also has column which is aggregation of values eg 0: [1.0.0.0.0.0.0.0.0]

In [3]:

#train_Y = np.asmatrix(enc.fit_transform(train_X['target'].reshape(-1,1)).toarray())
# test
# train_Y = pd.DataFrame(enc.fit_transform(train_X['target'].reshape(-1,1)).toarray()) #training for Y
# train_Y['aggr'] = train_Y.apply(lambda row: ''.join(map(str, row)), axis=1)
# in order to access eg the first row of the train_X use .iloc[0]
# to see all rows whose target is in range (0..9) use train_X.loc[trainX['target'] == 0]
# train_X.loc[train_X['target']== 0].sample(5)
# len(train_X.loc[train_X['target']== 1])

Get train and validation as subset from train

In [4]:
train_data = getdf('train')
# [whole:total rows in training dataset], [random: 10.000 random rows for validation],[train_random:the rest 50.000 for training]
whole = np.arange(train_data.shape[0]) 
random = np.random.randint(low=0, high=60000, size=10000)
train_random = np.delete(whole,[random])
# now make are validation and train dataframes
# validation dataframe
validation = train_data.iloc[random,:] #works
train = train_data.iloc[train_random,:]
train_Y = np.asmatrix(encoder.fit_transform(train['target'].reshape(-1,1)).toarray())
### now the test data 
test_X = getdf('test')
test_Y =  np.asmatrix(encoder.fit_transform(test_X['target'].reshape(-1,1)).toarray())

So we have VALIDATION, TRAIN, and then TEST

In [5]:
# validation
validation.sample(2)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,target
2800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [6]:
# train
train.sample(2)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,target
169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3512,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


get our X for logistic regression, insert 1 in front of each row of flattened digit, and init params

In [7]:
# X is train_X.ix[:,:784] and Y is train_X['target']
# X = train_X.ix[:,:784]


# add 1 infront of X
# X.insert(0,'ones',1)
# t = train['target']
#t = train_X['target']
# N,D = X.shape
# set params
#winit=0.01*np.random.normal(785,1)
#winit=0.01*np.random.rand(785,1)
# winit = np.ones(D)


NameError: name 'X' is not defined

In [ ]:
def sigmoid(z):
    yx = 1.0/(1+np.exp(-z))
    return yx

def ml_logregTrain(t,X,l,winit,options):
    w = winit
    iterations = options[0]
    tol = options[1]
    eta = options[2]
    ewold = float('-inf')
    for it in range(0,iterations):
        yx = np.dot(X.T,w)
        print("y(x) shape {}".format(yx.shape))
        s = sigmoid(yx)
        ew = sum(np.dot(t,s)) - np.log(sum(np.exp(s)))
        print('Iteration: {}, Cost function: {}'.format(it,ew))
        if np.absolute(ew -ewold) < tol:
            break
        gradient = np.matmul((t-s).T,X) - np.multiply(l,w)
        print(gradient.shape,eta)
        #w += np.multiply(np.float(eta),gradient)
        w += np.multiply(np.float(eta),np.reshape(gradient,(1)))
        ewold=ew
    ew =0
    return w 

In [ ]:
X = train.ix[:,:784]
X.insert(0,'ones',1)
t = train['target']
#t = train_X['target']
N,D = X.shape
# attempt to normalize X 
X = X.apply(lambda x: x/255)
winit=0.01*np.random.normal(size=(X.shape[0],785))
l = 0.5
options = [500,1/1000000 ,0.1/N]
w = ml_logregTrain(train_Y,X,l,winit,options)

In [ ]:
train_Y.shape

In [ ]:
winit.shape